# Deploying A Sentence-Embedding Model

## Contents

1. [Setup](#Setup)
2. [Deploy](#Deploy)
3. [Evaluate](#Evaluate)
4. [Update](#UPDATE)

# Setup

- Load environmental variables
- Create Sagemaker Session
- Define S3 Bucket, path and model name
- Provide IAM role

In [47]:
%load_ext dotenv
import json
import os
import sagemaker
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer
from sagemaker.pytorch import PyTorchModel

%dotenv -v

sagemaker_session = sagemaker.Session()

bucket = os.getenv("S3_BUCKET")
model_location = "{0}{1}".format(bucket, os.getenv("MODEL_PATH"))   # path to model data
entry = os.getenv("ENTRY_POINT")     # Python file to execute
source = os.getenv("SOURCE_DIR")       # Source for code
version = os.getenv("FRAMEWORK")           # Pytorch Version
role = sagemaker.get_execution_role()      # IAM role

git_conf = {'repo': 'https://github.com/sam3ay/sentence-compare.git', 'branch': 'code'}
git_config = {'repo': os.getenv("REPO"), 'branch': os.getenv("BRANCH")}

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# Deploy

Provide custom hosting functions:
- input_fn: Deserializes input data
- model_fn: Loads pytorch model
- predict_fn: run inference with input data
- output_fn: Serializes results of predict_fn
- predictor_cls: Creates the predictor with the endpoint name

In [28]:
class JSONPredictor(RealTimePredictor):
    """Serializes and deserializes json
    """
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [29]:
model = PyTorchModel(model_data=model_location,
                     role=role,
                     framework_version=version,
                     entry_point=entry,
                     source_dir=source,
                     predictor_cls=JSONPredictor,
                     git_config=git_conf)

In [31]:
predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.m5.xlarge',
                         endpoint_name="sentence-similarity")

Using already existing model: pytorch-inference-2020-05-28-23-53-02-991


---------------!

In [26]:
print(model_location)

s3://sentencetransformers/RoBERTa.tar.gz


# Evaluate

Determine Textual Similarity

In [45]:
address = {
    "query": ["Speak to people in a formal way."],
    "corpus": [
        "the particulars of the place where someone lives or an organization is situated.",
        "a formal speech delivered to an audience.",
        "write the name and address of the intended recipient on (an envelope, letter, or package).",
        "speak to (a person or an assembly), typically in a formal way.",
    ],
}

jsonAddress = json.dumps(address)
jsonOut = json.loads(jsonAddress)

print(jsonAddress)
print(jsonOut["corpus"])

response = predictor.predict(jsonAddress)
print(response)

{"query": ["Speak to people in a formal way"], "corpus": ["the particulars of the place where someone lives or an organization is situated.", "a formal speech delivered to an audience.", "write the name and address of the intended recipient on (an envelope, letter, or package).", "speak to (a person or an assembly), typically in a formal way."]}
['the particulars of the place where someone lives or an organization is situated.', 'a formal speech delivered to an audience.', 'write the name and address of the intended recipient on (an envelope, letter, or package).', 'speak to (a person or an assembly), typically in a formal way.']


ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "string indices must be integers". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/sentence-similarity in account 605622708578 for more information.

In [50]:
sagemaker_session.delete_endpoint(predictor.endpoint)